In [1]:
import sys
#import os
import pickle
import pandas as pd
import numpy as np
from pyarrow import feather
from scipy import sparse
from queue import Queue
from tqdm import tqdm
import itertools
#import concurrent
import multiprocessing

In [2]:
knn_arr = np.load("knn_ticker.npy")

In [3]:
from firebase_helper import *
import numpy as np

In [4]:
fb = firebase_helper()
db = fb.get_db()

In [5]:
db

In [6]:
ticker_map_path = "/mnt/d/data/news/ticker_maping_dict.pkl"
ticker_map_dict = pickle.load(open(ticker_map_path, "rb"))

In [ ]:
#overwrite ticker mapping on db
doc_ref = db.collection('ticker_map').document('dict')
doc_ref.set(ticker_map_dict)

In [ ]:
knn_ref = db.collection("knn").document("knn_ticker")

In [ ]:
knn_data = np.load("knn_ticker.npy")

In [ ]:
from typing import List
import os
from google.cloud.firestore_v1 import batch
from google.cloud.firestore_v1.batch import WriteBatch
from google.cloud.firestore_v1.client import Client
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

max_cpus = 12  # Maximum number of CPUs to use
batch_size = 500  # Batch size for Firestore writes

def insert_data(batch: batch, i: int) -> None:
    lc = knn_data[i]
    temp_map = {str(j): lc[j] for j in range(len(lc)) if lc[j] != 0}
    batch.set(db.collection('knn_news').document(str(i)), temp_map)

def process_batch(batch: WriteBatch, indices: List[int]) -> None:
    for i in indices:
        insert_data(batch, i)

with ThreadPoolExecutor(max_workers=min(max_cpus, os.cpu_count())) as executor:
    for i in tqdm(range(0, len(knn_data), batch_size)):
        batch = db.batch()
        process_batch(batch, range(i, min(i + batch_size, len(knn_data))))
        batch.commit()


In [ ]:
n = len(knn_data)
base_user_pref_vector = np.zeros(n)

In [24]:
def memoize(function):
    cache = {}
    def wrapper(input):
        if input not in cache:
            cache[input] = function(input)
        return cache[input]
    return wrapper

@memoize
def sic_match(input):
    sic_codes = {
        '01': 'Agriculture, Forestry, Fishing',
        '02': 'Agriculture, Forestry, Fishing',
        '07': 'Agriculture, Forestry, Fishing',
        '08': 'Agriculture, Forestry, Fishing',
        '09': 'Agriculture, Forestry, Fishing',
        '10': 'Mining',
        '11': 'Mining',
        '12': 'Mining',
        '13': 'Mining',
        '14': 'Mining',
        '15': 'Construction',
        '16': 'Construction',
        '17': 'Construction',
        **{f"{i:02d}": "Manufacturing" for i in range(20, 40)},
        **{f"{i:02d}": "Transportation & Public Utilities" for i in range(40, 50)},
        '50': 'Wholesale Trade',
        '51': 'Wholesale Trade',
        **{f"{i:02d}": "Retail Trade" for i in range(52, 60)},
        **{f"{i:02d}": "Finance, Insurance, Real Estate" for i in range(60, 68)},
        **{f"{i:02d}": "Services" for i in range(70, 90)},
        **{f"{i:02d}": "Public Administration" for i in range(91, 100)},
    }
    try:
        return sic_codes[input]
    except KeyError:
        raise ValueError("Invalid input. Please enter a two-character string matching a valid SIC code.")

In [ ]:
print(sic_match("01"))

In [7]:
#base imports
import os
import sys
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)
poly_dir = os.path.abspath(os.path.join(os.getcwd(), 'data_poly'))
sys.path.append(poly_dir)
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import itertools

# package imports
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
from tqdm import tqdm
import pyarrow.feather as feather

# local imports
import data_poly.poly_getdata as poly_getdata
import data_poly.poly_url as poly_url
import data_poly.poly_helper as poly_helper

In [8]:
from dotenv import load_dotenv
load_dotenv(dotenv_path="/home/peterzerg/repos/PeterZergQuant/.ENV")
api_key = os.environ.get("POLYGON_APIKEY_MASTER")

In [47]:
url_factory = poly_url.StockUrlFactory(api_key)
ticker_lc = ticker_map_dict.keys()
urls_dict = {ticker: url_factory.ReferenceData.ticker_info(url_factory, ticker) for ticker in ticker_lc}
df_dict = await poly_helper.get_data_from_urls(urls_dict)
df = pd.concat(df_dict.values())

In [48]:
sic_map_dict = {
    str(ticker): sic_match(sic_code[:2]) if isinstance(sic_code, str) else None
    for ticker, sic_code in zip(df.ticker, df.sic_code)
}

In [45]:
#upload data to db
doc_ref = db.collection('ticker_sic_map').document('dict')
doc_ref.set(sic_map_dict)

update_time {
  seconds: 1679289407
  nanos: 860875000
}

In [50]:
db.collection("user_preferences")